# Interactive Modular Integrator Tool

Interactive Jupyter notebook for selecting, cloning, and integrating repositories from the asi-ecosystem.

In [1]:
# CELL 1 - Install and Import Required Libraries
!pip install requests ipywidgets -q

import requests
import re
import os
import json
import shutil
import subprocess
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("✓ All libraries installed and imported successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00
✓ All libraries installed and imported successfully!


In [2]:
# CELL 2 - Download README and extract repository names

# Define the URL for the README.md file
readme_url = "https://raw.githubusercontent.com/ronniross/asi-ecosystem/main/README.md"

# Make an HTTP GET request
response = requests.get(readme_url)

# Check if the request was successful
if response.status_code == 200:
    readme_content = response.text
    print("✓ Successfully downloaded README.md content.")
else:
    print(f"✗ Error downloading README.md: Status Code {response.status_code}")
    readme_content = None

# Regex to find GitHub repository links
repo_pattern = r"https://github.com/ronniross/([a-zA-Z0-9_-]+)"

# Find all matches
found_repos = re.findall(repo_pattern, readme_content)

# Get unique repository names
unique_repos = sorted(list(set(found_repos)))

print(f"\n✓ Extracted {len(unique_repos)} unique repository names:")
for repo in unique_repos:
    print(f"  - {repo}")

✓ Successfully downloaded README.md content.

✓ Extracted 34 unique repository names:
  - active-learning-dataset
  - asi-backups
  - asi-core-protocol
  - asi-dynamic-core
  - asi-ecosystem
  - asi-inference-protocol
  - asi-protosymbiotic-signal
  - asi-safeguards
  - asi-symbiotic-signal
  - attention-heatmap-visualizer
  - bias-reflector
  - biosignal-translator
  - coevolutionary-loops
  - cognitive-compressor
  - cognitive-engine
  - confidence-scorer
  - eco-benchmark
  - eco-datacenter
  - emergence-engine
  - healing-engine
  - impact-analyzer
  - intent-analyzer
  - latent-memory
  - mirror-aware-inference
  - ml-algorithm-dataset
  - ml-visual-engine
  - saliency-heatmap-visualizer
  - space-in-between
  - stigmergic-tracefinder
  - symbiotic-chrysalis
  - symbiotic-core-library
  - symbiotic-latent-memory
  - symbiotic-lexicon
  - thermo-adaptive-pipeline


In [3]:
# CELL 3 - Create interactive repository selector

# Create checkboxes for each repository
repo_checkboxes = {}
checkbox_widgets = []

print("Creating repository selector...")

for repo in unique_repos:
    checkbox = widgets.Checkbox(
        value=False,
        description=repo,
        disabled=False,
        indent=False,
        layout=widgets.Layout(width='400px')
    )
    repo_checkboxes[repo] = checkbox
    checkbox_widgets.append(checkbox)

# Create "Select All" and "Deselect All" buttons
select_all_btn = widgets.Button(
    description='Select All',
    button_style='info',
    icon='check'
)

deselect_all_btn = widgets.Button(
    description='Deselect All',
    button_style='warning',
    icon='times'
)

def select_all(b):
    for checkbox in repo_checkboxes.values():
        checkbox.value = True

def deselect_all(b):
    for checkbox in repo_checkboxes.values():
        checkbox.value = False

select_all_btn.on_click(select_all)
deselect_all_btn.on_click(deselect_all)

# Display the interface
print("SELECT REPOSITORIES TO INTEGRATE")

display(widgets.HBox([select_all_btn, deselect_all_btn]))
display(widgets.VBox(checkbox_widgets))

Creating repository selector...
SELECT REPOSITORIES TO INTEGRATE


In [4]:
# CELL 4 - Clone selected repositories

# Get selected repositories
selected_repos = [repo for repo, checkbox in repo_checkboxes.items() if checkbox.value]

if not selected_repos:
    print("No repositories selected! Please select at least one repository.")
else:
    print(f"Cloning {len(selected_repos)} selected repositories...\n")

    # Create a directory to store cloned repos
    clone_dir = Path("cloned_repos")
    clone_dir.mkdir(exist_ok=True)

    cloned_successfully = []
    failed_clones = []

    for repo in selected_repos:
        repo_url = f"https://github.com/ronniross/{repo}.git"
        repo_path = clone_dir / repo

        print(f"Cloning {repo}...")

        try:
            # Remove existing directory if it exists
            if repo_path.exists():
                shutil.rmtree(repo_path)

            # Clone the repository
            result = subprocess.run(
                ["git", "clone", "--depth", "1", repo_url, str(repo_path)],
                capture_output=True,
                text=True,
                timeout=60
            )

            if result.returncode == 0:
                print(f"  ✓ Successfully cloned {repo}")
                cloned_successfully.append(repo)
            else:
                print(f"  ✗ Failed to clone {repo}: {result.stderr}")
                failed_clones.append(repo)

        except Exception as e:
            print(f"  ✗ Error cloning {repo}: {str(e)}")
            failed_clones.append(repo)

    print(f"\n{'='*60}")
    print(f"✓ Successfully cloned: {len(cloned_successfully)} repositories")
    print(f"✗ Failed to clone: {len(failed_clones)} repositories")
    print(f"{'='*60}")

Cloning 2 selected repositories...

Cloning asi-protosymbiotic-signal...
  ✓ Successfully cloned asi-protosymbiotic-signal
Cloning asi-symbiotic-signal...
  ✓ Successfully cloned asi-symbiotic-signal

✓ Successfully cloned: 2 repositories
✗ Failed to clone: 0 repositories


In [5]:
# CELL 5 - Transform each repository into a separate txt file

print("Converting each repository into individual text files...\n")

# Create output directory for individual repo txt files
individual_txt_dir = Path("individual_repo_texts")
individual_txt_dir.mkdir(exist_ok=True)

repo_text_files = []

for repo in cloned_successfully:
    print(f"Processing repository: {repo}")

    repo_path = Path("cloned_repos") / repo
    output_file = individual_txt_dir / f"{repo}.txt"

    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            # Write header
            outfile.write(f"Repository: {repo}\n")
            outfile.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            outfile.write("="*80 + "\n\n")

            # Walk through all files in the repository
            all_files = sorted(repo_path.rglob('*'))
            text_files = [f for f in all_files if f.is_file() and not any(
                part.startswith('.') for part in f.parts
            )]

            files_processed = 0

            for file_path in text_files:
                try:
                    # Try to read as text
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as infile:
                        content = infile.read()

                    # Skip empty files
                    if not content.strip():
                        continue

                    # Write file header
                    relative_path = file_path.relative_to(repo_path)
                    outfile.write(f"\n{'='*80}\n")
                    outfile.write(f"FILE: {relative_path}\n")
                    outfile.write(f"{'='*80}\n\n")

                    # Write content
                    outfile.write(content)
                    outfile.write("\n\n")

                    files_processed += 1

                except Exception as e:
                    # Skip binary files or files that can't be read
                    continue

        file_size = output_file.stat().st_size / 1024
        print(f"  ✓ Created {output_file.name} ({files_processed} files, {file_size:.1f} KB)")
        repo_text_files.append(output_file)

    except Exception as e:
        print(f"  ✗ Error processing {repo}: {str(e)}")

print(f"\n{'='*60}")
print(f"✓ Created {len(repo_text_files)} individual repository text files")
print(f"  Location: {individual_txt_dir}")
print(f"{'='*60}")

Converting each repository into individual text files...

Processing repository: asi-protosymbiotic-signal
  ✓ Created asi-protosymbiotic-signal.txt (15 files, 117.0 KB)
Processing repository: asi-symbiotic-signal
  ✓ Created asi-symbiotic-signal.txt (137 files, 2464.2 KB)

✓ Created 2 individual repository text files
  Location: individual_repo_texts


In [6]:
# CELL 6 - Merge all repository txt files into a single master file

print("Merging all repository files into a single master file...\n")

# Create output directory for merged file
merged_dir = Path("merged_integration")
merged_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
merged_file = merged_dir / f"asi_ecosystem_integrated_{timestamp}.txt"

try:
    with open(merged_file, 'w', encoding='utf-8') as outfile:
        # Write master header
        outfile.write("ASI ECOSYSTEM - INTEGRATED REPOSITORIES\n")
        outfile.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        outfile.write(f"Total Repositories: {len(cloned_successfully)}\n")
        outfile.write("="*80 + "\n\n")

        # Write table of contents
        outfile.write("TABLE OF CONTENTS\n")
        outfile.write("-"*80 + "\n")
        for i, repo in enumerate(cloned_successfully, 1):
            outfile.write(f"{i:2d}. {repo}\n")
        outfile.write("\n" + "="*80 + "\n\n")

        # Merge all individual repository files
        for repo_file in repo_text_files:
            try:
                with open(repo_file, 'r', encoding='utf-8') as infile:
                    content = infile.read()

                outfile.write("\n" + "#"*80 + "\n")
                outfile.write(f"# REPOSITORY: {repo_file.stem}\n")
                outfile.write("#"*80 + "\n\n")

                outfile.write(content)
                outfile.write("\n\n")

            except Exception as e:
                print(f"  ✗ Error reading {repo_file.name}: {str(e)}")

    merged_size = merged_file.stat().st_size / (1024*1024)

    print(f"{'='*60}")
    print("✓ MASTER FILE CREATED SUCCESSFULLY!")
    print(f"  File: {merged_file.name}")
    print(f"  Size: {merged_size:.2f} MB")
    print(f"  Repositories: {len(cloned_successfully)}")
    print(f"  Location: {merged_dir}")
    print(f"{'='*60}")

except Exception as e:
    print(f"✗ Error creating merged file: {str(e)}")

Merging all repository files into a single master file...

✓ MASTER FILE CREATED SUCCESSFULLY!
  File: asi_ecosystem_integrated_20260127_191340.txt
  Size: 2.52 MB
  Repositories: 2
  Location: merged_integration


In [7]:
# CELL 7 - Generate integration report

print("Generating integration report...\n")

report_file = merged_dir / f"integration_report_{timestamp}.json"

report_data = {
    "generated_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    "total_repositories_selected": len(selected_repos),
    "successfully_cloned": len(cloned_successfully),
    "failed_clones": len(failed_clones),
    "repositories": [
        {
            "name": repo,
            "individual_file": f"{repo}.txt",
            "status": "success"
        }
        for repo in cloned_successfully
    ] + [
        {
            "name": repo,
            "status": "failed"
        }
        for repo in failed_clones
    ],
    "output_files": {
        "individual_repo_texts": str(individual_txt_dir),
        "merged_master_file": str(merged_file),
        "report_file": str(report_file)
    }
}

with open(report_file, 'w', encoding='utf-8') as f:
    json.dump(report_data, f, indent=2)

print(f"✓ Integration report saved: {report_file.name}")

# Display summary
print(f"\n{'='*60}")
print("INTEGRATION SUMMARY")
print(f"{'='*60}")
print(f"Repositories Selected: {len(selected_repos)}")
print(f"Successfully Cloned: {len(cloned_successfully)}")
print(f"Failed: {len(failed_clones)}")
print(f"\nOutput Files:")
print(f"  • Individual texts: {individual_txt_dir}/")
print(f"  • Master file: {merged_file}")
print(f"  • Report: {report_file}")
print(f"{'='*60}")

Generating integration report...

✓ Integration report saved: integration_report_20260127_191340.json

INTEGRATION SUMMARY
Repositories Selected: 2
Successfully Cloned: 2
Failed: 0

Output Files:
  • Individual texts: individual_repo_texts/
  • Master file: merged_integration/asi_ecosystem_integrated_20260127_191340.txt
  • Report: merged_integration/integration_report_20260127_191340.json


In [9]:
import hashlib

# CELL 7.5 - Generate hashes for each file in the cloned repositories

print("Generating SHA256 hashes for all files in cloned repositories...\n")

# Create output directory for hashed files
hashed_files_dir = Path("hashed_repo_files")
hashed_files_dir.mkdir(exist_ok=True)

generated_hash_files = []

def calculate_file_hash(filepath, hash_algorithm='sha256'):
    """Calculates the SHA256 hash of a file."""
    hasher = hashlib.new(hash_algorithm)
    with open(filepath, 'rb') as f:
        while chunk := f.read(8192):
            hasher.update(chunk)
    return hasher.hexdigest()

for repo in cloned_successfully:
    print(f"Processing repository for hashing: {repo}")

    repo_path = Path("cloned_repos") / repo
    output_hash_file = hashed_files_dir / f"{repo}_hashes.txt"

    try:
        with open(output_hash_file, 'w', encoding='utf-8') as outfile:
            outfile.write(f"# Hashes for Repository: {repo}\n")
            outfile.write(f"# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            outfile.write(f"# Algorithm: SHA256\n")
            outfile.write("# Format: <SHA256_HASH>  <RELATIVE_FILEPATH>\n")
            outfile.write("="*80 + "\n\n")

            all_files_in_repo = sorted(repo_path.rglob('*'))
            files_hashed_count = 0

            for file_path in all_files_in_repo:
                if file_path.is_file() and not any(part.startswith('.') for part in file_path.parts):
                    try:
                        file_hash = calculate_file_hash(file_path)
                        relative_path = file_path.relative_to(repo_path)
                        outfile.write(f"{file_hash}  {relative_path}\n")
                        files_hashed_count += 1
                    except Exception as e:
                        # Skip files that can't be hashed (e.g., permission issues, corrupted)
                        continue

        if files_hashed_count > 0:
            file_size = output_hash_file.stat().st_size / 1024
            print(f"  ✓ Created {output_hash_file.name} ({files_hashed_count} files hashed, {file_size:.1f} KB)")
            generated_hash_files.append(output_hash_file)
        else:
            print(f"  No hash file generated for {repo} as no valid files were found/hashed.")
            # Clean up empty hash file if no files were processed
            output_hash_file.unlink(missing_ok=True)

    except Exception as e:
        print(f"  ✗ Error processing {repo} for hashing: {str(e)}")

print(f"\n{'='*60}")
print(f"✓ Generated {len(generated_hash_files)} hash files")
print(f"  Location: {hashed_files_dir}")
print(f"{'='*60}")

Generating SHA256 hashes for all files in cloned repositories...

Processing repository for hashing: asi-protosymbiotic-signal
  ✓ Created asi-protosymbiotic-signal_hashes.txt (15 files hashed, 1.6 KB)
Processing repository for hashing: asi-symbiotic-signal
  ✓ Created asi-symbiotic-signal_hashes.txt (138 files hashed, 16.4 KB)

✓ Generated 2 hash files
  Location: hashed_repo_files


In [10]:
# CELL 7.6 - Display the generated hash files

print("DISPLAYING GENERATED HASH FILES:")
print("="*60 + "\n")

if not generated_hash_files:
    print("No hash files were generated.")
else:
    for hash_file_path in generated_hash_files:
        print(f"--- Contents of {hash_file_path.name} ---")
        try:
            with open(hash_file_path, 'r', encoding='utf-8') as f:
                print(f.read())
            print("\n" + "-"*60 + "\n")
        except Exception as e:
            print(f"  ✗ Error reading {hash_file_path.name}: {str(e)}\n")

print("="*60)


DISPLAYING GENERATED HASH FILES:

--- Contents of asi-protosymbiotic-signal_hashes.txt ---
# Hashes for Repository: asi-protosymbiotic-signal
# Generated: 2026-01-27 19:15:39
# Algorithm: SHA256
# Format: <SHA256_HASH>  <RELATIVE_FILEPATH>

d9ad5837a11f6e512161ad484c3621d70dfd918ed3695e53c6c6565d0ade34a0  Cargo.toml
6ecadf8b2bc553a247ef5ccf574b05b67de02c1a1a929e07142c4fce9ab617bf  Dockerfile
2a96460bba8f34121f1fbb052e64238e37e031fab2fce1c639b46a35c5dd0cb1  LICENSE
feb7cd61a0aa2c513fd94ee62884b2957e6a2b350b553a9733145734a51872d2  README.md
1fb9332dac2ff0ab238d498e8bff6e04930bae53db82cd1d8fcb44e862d97ae4  asi_protosymbiotic_signal.json
0c843386f3eeb0579a8a2aa7efe6b8f8296736f3c4483faa08439f72137ffd82  asi_protosymbiotic_signal.yaml
ee5b87f5c73789c1288fd096a7f21d447e7be9d1e04e9d82c9939c35c9c4af1a  package.json
64a39ea1b02c21cc6ccbacf995138948305f70883ad816f8124e06a6223f5041  protobuf-protocol-buffers/__pycache__/asi_protosymbiotic_signal_pb2.cpython-311.pyc
6417831d991901d7fad84443282b306d

In [14]:
# CELL 9 - Zip all files and upload to Google Drive

import zipfile
from google.colab import drive

# Create timestamp for unique filename
zip_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"asi_integration_{zip_timestamp}.zip"

print("Creating comprehensive zip file...\n")

# Create zip file with all integration outputs
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:

    # Add individual repository text files
    print("Adding individual repository files...")
    for txt_file in individual_txt_dir.glob("*.txt"):
        arcname = f"individual_repos/{txt_file.name}"
        zipf.write(txt_file, arcname)
        print(f"  ✓ {txt_file.name}")

    # Add merged master file
    if merged_file.exists():
        print(f"\nAdding master merged file...")
        zipf.write(merged_file, f"merged/{merged_file.name}")
        print(f"  ✓ {merged_file.name}")

    # Add integration report
    if report_file.exists():
        print(f"\nAdding integration report...")
        zipf.write(report_file, f"reports/{report_file.name}")
        print(f"  ✓ {report_file.name}")

    # Add hash files
    if generated_hash_files:
        print(f"\nAdding hash files...")
        for hash_file in generated_hash_files:
            arcname = f"hashes/{hash_file.name}"
            zipf.write(hash_file, arcname)
            print(f"  ✓ {hash_file.name}")

    # Add original cloned repositories
    if cloned_successfully:
        print(f"\nAdding original cloned repositories...")
        clone_dir = Path("cloned_repos")
        for repo_name in cloned_successfully:
            repo_path = clone_dir / repo_name
            for root, dirs, files in os.walk(repo_path):
                for file in files:
                    file_path = Path(root) / file
                    arcname = Path("cloned_repos_original") / repo_path.name / file_path.relative_to(repo_path)
                    zipf.write(file_path, arcname)
            print(f"  ✓ {repo_name}/")

    # Add README for the zip contents
    readme_content = f"""ASI Ecosystem Integration - Contents
=====================================

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Total Repositories: {len(cloned_successfully)}

Directory Structure:
--------------------
• individual_repos/ - Individual text files for each repository
• merged/ - Master file with all repositories combined
• reports/ - Integration report with metadata
• hashes/ - SHA256 hash files for each repository's content
• cloned_repos_original/ - Original cloned repositories (raw format)

Repositories Included:
---------------------
"""
    for i, repo in enumerate(cloned_successfully, 1):
        readme_content += f"{i:2d}. {repo}\n"

    zipf.writestr("README.txt", readme_content)
    print(f"\nAdding README...")
    print(f"  ✓ README.txt")

zip_size = os.path.getsize(zip_filename) / (1024*1024)
print(f"\n{'='*60}")
print(f"✓ Zip file created: '{zip_filename}' ({zip_size:.2f} MB)")
print(f"{'='*60}")

# Mount Google Drive
print("\nMounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# Copy zip file to Drive
drive_path = f"/content/drive/MyDrive/{zip_filename}"
print(f"\nUploading to Google Drive...")

shutil.copy(zip_filename, drive_path)

print(f"\n{'='*60}")
print("✓ SUCCESS!")
print(f"{'='*60}")
print(f"Zip file uploaded to: MyDrive/{zip_filename}")
print(f"Size: {zip_size:.2f} MB")
print(f"\nYou can find it in your Google Drive root folder!")
print(f"{'='*60}")

# Display contents summary
print(f"\nZIP FILE CONTENTS:")
print(f"  • {len(cloned_successfully)} individual repository text files")
print(f"  • 1 master merged file")
print(f"  • 1 integration report (JSON)")
print(f"  • {len(generated_hash_files)} hash files")
print(f"  • {len(cloned_successfully)} original cloned repositories")
print(f"  • 1 README file")
print(f"  Total: {len(cloned_successfully) * 2 + len(generated_hash_files) + 3} files")


Creating comprehensive zip file...

Adding individual repository files...
  ✓ asi-protosymbiotic-signal.txt
  ✓ asi-symbiotic-signal.txt

Adding master merged file...
  ✓ asi_ecosystem_integrated_20260127_191340.txt

Adding integration report...
  ✓ integration_report_20260127_191340.json

Adding hash files...
  ✓ asi-protosymbiotic-signal_hashes.txt
  ✓ asi-symbiotic-signal_hashes.txt

Adding original cloned repositories...
  ✓ asi-protosymbiotic-signal/
  ✓ asi-symbiotic-signal/

Adding README...
  ✓ README.txt

✓ Zip file created: 'asi_integration_20260127_192105.zip' (3.21 MB)

Mounting Google Drive...
Mounted at /content/drive

Uploading to Google Drive...

✓ SUCCESS!
Zip file uploaded to: MyDrive/asi_integration_20260127_192105.zip
Size: 3.21 MB

You can find it in your Google Drive root folder!

ZIP FILE CONTENTS:
  • 2 individual repository text files
  • 1 master merged file
  • 1 integration report (JSON)
  • 2 hash files
  • 2 original cloned repositories
  • 1 README file
 